In [17]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [50]:
world_bank = pd.read_csv("data/SDG_csv/SDGData.csv", sep=",")

In [40]:
life_expectancy = pd.read_csv("data/life-expectancy.csv")

In [45]:
household_ppp = pd.read_excel("data/NHA-indicators.xlsx")

In [51]:
#Zoek machine voor onderwerpen uit de indicator naam. Niet hoofdletter gevoelig. 
#Vergeet niet country_names te laden.

variabelen = []
for sentence in list(world_bank["Indicator Name"].values):
    sentence1 = np.array(sentence.split(" "))
    for words in (sentence1):
        if words.lower() == "GDP".lower(): #Vul hier zoekterm in.
            variabelen += [sentence]

variabelen = pd.unique(variabelen)
print(variabelen)                       
                                


['GDP (constant 2010 US$)' 'GDP (constant LCU)' 'GDP (current LCU)'
 'GDP (current US$)' 'GDP growth (annual %)'
 'GDP per capita (constant 2010 US$)' 'GDP per capita (constant LCU)'
 'GDP per capita (current LCU)' 'GDP per capita (current US$)'
 'GDP per capita growth (annual %)'
 'GDP per capita, PPP (constant 2011 international $)'
 'GDP per capita, PPP (current international $)'
 'GDP per person employed (constant 2011 PPP $)'
 'PPP conversion factor, GDP (LCU per international $)'
 'Water productivity, total (constant 2010 US$ GDP per cubic meter of total freshwater withdrawal)']


In [82]:
countries = world_bank[world_bank["Indicator Name"]== "GDP (current US$)"]["Country Name"]
GDP_2017 = world_bank[world_bank["Indicator Name"]== "GDP (current US$)"]["2017"]

GDP = countries.to_frame().join(GDP_2017.to_frame())
GDP[GDP["2017"] == max(GDP["2017"])]


,Country Name,2017
16509,World,8.068379e+13
